<h1 align='center'>Evaluation</h1>

In [8]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle
import faiss

with open('overview_embeddings.pkl', 'rb') as f:
        embeddings = pickle.load(f)

# Mock test set
test_embeddings = embeddings[:1000]

In [9]:
test_embeddings

array([[-0.10088539,  0.07117508, -0.0289718 , ..., -0.01160109,
         0.0589892 , -0.06030006],
       [-0.02430348,  0.00925271, -0.06100011, ..., -0.02184225,
         0.05695461,  0.01922566],
       [-0.00636554,  0.06156255,  0.03046027, ...,  0.00987918,
         0.03299569,  0.00213429],
       ...,
       [-0.03863252, -0.01697408, -0.00917226, ..., -0.04266263,
        -0.01877826,  0.02456542],
       [ 0.01505129,  0.09284832, -0.05677039, ...,  0.02402589,
         0.08759901, -0.02167096],
       [-0.02143573,  0.0115293 , -0.0339081 , ...,  0.07279328,
        -0.00549321, -0.0741011 ]], dtype=float32)

In [10]:
import pandas as pd
df = pd.read_csv('cleaned_data.csv')

In [11]:
# Build quantized index

index = faiss.IndexIVFPQ(
    faiss.IndexFlatIP(384),   # Inner product metric
    384,                      # Dimention
    1024,                     # Number of Voronoi Cells (nlist)
    64,                       # Subquantizers (m)
    8                         # Bits per subquantizers
)

In [12]:
# Ground Truth Generator

def generate_ground_truth(query_idx, k=5):
    """Based on genre similarity as proxy"""
    genres = df['genres'].iloc[query_idx]
    similar = df[df['genres'].str.contains(genres.split(',')[0], regex=False)].index.tolist()
    return similar[:k]

In [13]:
# Precision @ k calculator

def precision_at_k(query_idx, recommendations, k=5):
    true_positives = set(generate_ground_truth(query_idx)) & set(recommendations[:k])
    return len(true_positives) / k

In [18]:
# Test evaluation
query_index = 27
_, indices = index.search(test_embeddings[query_index:query_index+1], 10)
print(f"Precision@5: {precision_at_k(query_index, indices[0])}")

Precision@5: 0.0
